# Library Import

In [1]:
# import os
import json
import warnings
warnings.filterwarnings(action='ignore')

from pycocotools.coco import COCO
import numpy as np
from tqdm import tqdm
# import cv2


import torch
# import torch.nn as nn
# from torch.nn import functional as F
from torch.utils.data import Dataset

# import torchvision
# from torchvision.models import vgg16
# from torchvision.ops import RoIPool
# from torchvision.ops import nms
# from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# from torchnet.meter import ConfusionMeter, AverageValueMeter


In [5]:
## Edit freely
annotation = '/opt/ml/detection/dataset/train.json' # annotation directory
data_dir = '/opt/ml/detection/dataset'    #data image directory
check_mode = True    #if true, picked label is printed(0:10)
save_dir = '/opt/ml/personel/easy_train_testver.json'
thr = 10
## --do not edit below--


# Helper functions

In [6]:
def read_json(dir):
    with open(dir, 'r') as f:
        json_data = json.load(f)

    return json_data


#레이블 갯수
def get_label(dataset, thr = 5):
    from collections import defaultdict
    from tqdm import tqdm
    idxlist = []

    for i in tqdm(range(len(dataset))):
        _, bbox, labels = dataset[i]
        bbox, labels = bbox.tolist(), labels.tolist()
        if len(labels) <= thr:
            idxlist.append(i)

    return idxlist


def editer(annotation, idxs, save_dir):
    train_dataset = read_json(annotation) 
    
    edited_list = []
    for ann in train_dataset['annotations']:
        if ann['image_id'] in idxs:
            edited_list.append(ann)
    train_dataset['annotations'] = edited_list

    edited_list = []
    for ann in train_dataset['images']:
        if ann['id'] in idxs:
            edited_list.append(ann)
    train_dataset['images'] = edited_list
    
    with open(save_dir, 'w', encoding='utf-8') as make_file:
        json.dump(train_dataset, make_file, indent="\t")
    
    print(f'done! file_dir: {save_dir}')
    return None

In [7]:
# TrainDataset
class TrainCustom(Dataset):
    def __init__(self, annotation, data_dir, transforms = False):
        """
        Args:
            annotation: annotation 파일 위치
            data_dir: data가 존재하는 폴더 경로
            transforms : transform or not
        """

        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.transforms = transforms

    def __getitem__(self, index: int):
        
        # 이미지 아이디 가져오기
        image_id = self.coco.getImgIds(imgIds=index)

        # 이미지 정보 가져오기
        image_info = self.coco.loadImgs(image_id)[0]

        # 이미지 로드
        #image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = None

        # 어노테이션 파일 로드
        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        # 박스 가져오기
        boxes = np.array([x['bbox'] for x in anns])

        # boxes (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]

        # 레이블 가져오기
        labels = np.array([x['category_id'] for x in anns])
        labels = torch.as_tensor(labels, dtype=torch.int64)

        sample = {
            'image': image,
            'bboxes': boxes,
            'labels': labels
        }

        bboxes = torch.tensor(sample['bboxes'], dtype=torch.float32)
        boxes = torch.tensor(sample['bboxes'], dtype=torch.float32)

        # bboxes (x_min, y_min, x_max, y_max) -> boxes (y_min, x_min, y_max, x_max)
        boxes[:, 0] = bboxes[:, 1]
        boxes[:, 1] = bboxes[:, 0]
        boxes[:, 2] = bboxes[:, 3]
        boxes[:, 3] = bboxes[:, 2]


        # final boxes: (y_min, x_min, y_max, x_max)
        return image, boxes, labels

    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [8]:
dataset = TrainCustom(annotation, data_dir, transforms=True)
idxs = get_label(dataset, thr)
if check_mode:
    print(idxs[0:10])
else:
    print(f'check mode {check_mode}')

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


100%|██████████| 4883/4883 [00:00<00:00, 9927.14it/s]

[0, 1, 2, 3, 4, 6, 7, 8, 9, 10]


In [9]:
editer(annotation, idxs, save_dir)

done! file_dir: /opt/ml/personel/easy_train_testver.json
